# NumberExplorer: Perfect Numbers Investigation

An interactive exploration of perfect numbers, focusing on even perfect numbers via the Euclid-Euler theorem and investigating potential odd perfect number candidates.

## Overview

A perfect number is a positive integer that equals the sum of its proper positive divisors (divisors excluding the number itself). For example:
- **6** is perfect: 1 + 2 + 3 = 6
- **28** is perfect: 1 + 2 + 4 + 7 + 14 = 28

## Mathematical Background

### Even Perfect Numbers
**Euclid-Euler Theorem**: An even number is perfect if and only if it has the form 2^(p-1) × (2^p - 1), where 2^p - 1 is a Mersenne prime.

### Odd Perfect Numbers
The existence of odd perfect numbers remains one of mathematics' greatest unsolved problems. If they exist, they must satisfy numerous constraints.

In [ ]:
# Import required libraries
import sympy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import concurrent.futures
from math import gcd, log10
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ NumberExplorer libraries imported successfully!")
print("🔢 Ready to explore the fascinating world of perfect numbers")

## Core Perfect Number Functions

Let's implement the core functions for exploring perfect numbers:

In [ ]:
def generate_perfect(p_max):
    """
    Generate even perfect numbers using Euclid–Euler theorem up to exponent p_max.
    Returns a list of tuples (p, perfect_number).
    """
    perfects = []
    for p in range(2, p_max + 1):
        mersenne = (1 << p) - 1  # 2^p - 1
        if sympy.isprime(mersenne):
            perfect = (1 << (p - 1)) * mersenne  # 2^(p-1) * (2^p - 1)
            perfects.append((p, perfect))
    return perfects

def validate_perfect(n):
    """
    Validate n is a perfect number by checking sum of proper divisors equals n.
    """
    if n < 2:
        return False
    divisors = sympy.divisors(n)
    proper_divisors = divisors[:-1]  # Exclude n itself
    return sum(proper_divisors) == n

def analyse_number_properties(n):
    """
    Analyse mathematical properties of a number.
    """
    divisors = sympy.divisors(n)
    proper_divisors = divisors[:-1]
    sigma = sum(divisors)  # Sum of all divisors including n
    sigma_proper = sum(proper_divisors)  # Sum of proper divisors
    
    properties = {
        'number': n,
        'divisors': divisors,
        'proper_divisors': proper_divisors,
        'divisor_count': len(divisors),
        'sigma': sigma,
        'sigma_proper': sigma_proper,
        'is_perfect': sigma_proper == n,
        'is_abundant': sigma_proper > n,
        'is_deficient': sigma_proper < n,
        'abundance': sigma_proper - n,
        'prime_factors': sympy.factorint(n)
    }
    
    return properties

def basic_conditions_odd(n):
    """
    Check if n satisfies basic odd perfect number necessary conditions.
    """
    if n <= 1 or n % 2 == 0:
        return False
    
    # Known necessary conditions for odd perfect numbers
    conditions = [
        n % 105 != 0,  # Not divisible by 105
        n % 12 == 1 or n % 468 == 117 or n % 324 == 81  # Modular conditions
    ]
    
    return all(conditions)

def prime_factor_filters(n):
    """
    Analyse prime factors of n for odd perfect number criteria.
    Returns True if filters pass.
    """
    factors = sympy.factorint(n)
    primes = list(factors.keys())
    
    if len(primes) < 2:  # Must have at least 2 distinct prime factors
        return False
    
    # Additional theoretical constraints
    if primes[0] < 100:  # Smallest prime factor constraint
        return False
    
    return True

# Test the functions
first_perfects = generate_perfect(20)
print(f"\n🎯 First few even perfect numbers:")
for p, perfect in first_perfects[:5]:
    print(f"p={p}: 2^{p-1} × (2^{p} - 1) = {perfect}")
    
# Validate the first perfect number
if first_perfects:
    test_perfect = first_perfects[0][1]
    props = analyse_number_properties(test_perfect)
    print(f"\n✅ Validation of {test_perfect}:")
    print(f"   Proper divisors: {props['proper_divisors']}")
    print(f"   Sum of proper divisors: {props['sigma_proper']}")
    print(f"   Is perfect: {props['is_perfect']}")

## Interactive Perfect Number Generator

Explore even perfect numbers interactively:

In [ ]:
# Create interactive perfect number generator
max_exponent_slider = widgets.IntSlider(
    value=15,
    min=2,
    max=31,  # Limited for computational reasons
    step=1,
    description='Max Exponent:',
    tooltip='Maximum exponent p to test (p ≤ 31 for performance)'
)

generate_button = widgets.Button(
    description='Generate Perfect Numbers',
    button_style='primary',
    tooltip='Generate even perfect numbers using Euclid-Euler theorem'
)

output_area = widgets.Output()

def on_generate_perfect_click(b):
    with output_area:
        clear_output()
        max_p = max_exponent_slider.value
        
        print(f"🔍 Searching for even perfect numbers up to p={max_p}...\n")
        
        start_time = time.time()
        perfects = generate_perfect(max_p)
        end_time = time.time()
        
        print(f"⏱️  Search completed in {end_time - start_time:.3f} seconds")
        print(f"📊 Found {len(perfects)} perfect numbers\n")
        
        if perfects:
            print("📋 Even Perfect Numbers Found:")
            print("=" * 60)
            print(f"{'#':>3} | {'p':>3} | {'Mersenne Prime':>15} | {'Perfect Number':>20}")
            print("-" * 60)
            
            for i, (p, perfect) in enumerate(perfects, 1):
                mersenne = (1 << p) - 1
                if perfect < 10**15:  # Show full number if not too large
                    print(f"{i:>3} | {p:>3} | {mersenne:>15} | {perfect:>20}")
                else:  # Use scientific notation for very large numbers
                    print(f"{i:>3} | {p:>3} | {mersenne:>15} | {perfect:.6e}")
            
            # Show detailed analysis for the largest reasonable perfect number
            if len(perfects) > 0:
                for p, perfect in perfects:
                    if perfect < 10**12:  # Analyse if not too large
                        props = analyse_number_properties(perfect)
                        print(f"\n🔬 Detailed Analysis of {perfect}:")
                        print(f"   Number of divisors: {props['divisor_count']}")
                        print(f"   Proper divisors: {props['proper_divisors'][:10]}{'...' if len(props['proper_divisors']) > 10 else ''}")
                        print(f"   Sum of all divisors (σ): {props['sigma']}")
                        print(f"   Prime factorisation: {props['prime_factors']}")
                        break
        else:
            print("❌ No perfect numbers found in the specified range.")
            print("   Try increasing the maximum exponent.")

generate_button.on_click(on_generate_perfect_click)

# Display the generator widget
perfect_generator_widget = widgets.VBox([
    widgets.HTML("<h2>🔢 Interactive Perfect Number Generator</h2>"),
    max_exponent_slider,
    generate_button,
    output_area
])

display(perfect_generator_widget)

# Trigger initial generation
on_generate_perfect_click(None)

## Perfect Number Visualisations

Visualise the growth and properties of perfect numbers:

In [ ]:
def create_perfect_number_visualisations(max_p=25):
    """Create comprehensive visualisations of perfect numbers."""
    perfects = generate_perfect(max_p)
    
    if not perfects:
        print("No perfect numbers found for visualisation.")
        return
    
    exponents, values = zip(*perfects)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Growth of Perfect Numbers (Log Scale)
    axes[0, 0].plot(exponents, values, 'bo-', linewidth=2, markersize=8)
    axes[0, 0].set_yscale('log')
    axes[0, 0].set_xlabel('Exponent p')
    axes[0, 0].set_ylabel('Perfect Number (log scale)')
    axes[0, 0].set_title('Growth of Even Perfect Numbers')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Add value labels for smaller numbers
    for i, (p, val) in enumerate(perfects[:6]):
        if val < 10**10:
            axes[0, 0].annotate(f'{val}', (p, val), 
                              textcoords="offset points", xytext=(0,10), ha='center')
    
    # 2. Number of Digits Growth
    num_digits = [len(str(val)) for val in values]
    axes[0, 1].bar(exponents, num_digits, color='orange', alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('Exponent p')
    axes[0, 1].set_ylabel('Number of Digits')
    axes[0, 1].set_title('Digital Length of Perfect Numbers')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Add value labels
    for i, (p, digits) in enumerate(zip(exponents, num_digits)):
        axes[0, 1].text(p, digits + 0.5, str(digits), ha='center', va='bottom')
    
    # 3. Mersenne Primes (the 2^p - 1 values)
    mersennes = [(1 << p) - 1 for p in exponents]
    axes[0, 2].plot(exponents, mersennes, 'ro-', linewidth=2, markersize=6)
    axes[0, 2].set_yscale('log')
    axes[0, 2].set_xlabel('Exponent p')
    axes[0, 2].set_ylabel('Mersenne Prime 2^p - 1 (log scale)')
    axes[0, 2].set_title('Mersenne Primes')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Ratio Between Consecutive Perfect Numbers
    if len(values) > 1:
        ratios = [values[i] / values[i-1] for i in range(1, len(values))]
        ratio_exponents = exponents[1:]
        
        axes[1, 0].plot(ratio_exponents, ratios, 'go-', linewidth=2, markersize=6)
        axes[1, 0].set_xlabel('Exponent p')
        axes[1, 0].set_ylabel('Ratio P(p) / P(p-1)')
        axes[1, 0].set_title('Growth Ratios Between Perfect Numbers')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Add ratio labels
        for i, (p, ratio) in enumerate(zip(ratio_exponents, ratios)):
            if i < 8:  # Limit labels to avoid crowding
                axes[1, 0].annotate(f'{ratio:.1f}', (p, ratio),
                                  textcoords="offset points", xytext=(0,10), ha='center')
    
    # 5. Perfect Number Classification (showing first few with their properties)
    small_perfects = [(p, val) for p, val in perfects if val < 10**8]  # Analyse smaller ones
    
    if small_perfects:
        perfect_data = []
        for p, perfect in small_perfects[:6]:  # Limit to first 6
            props = analyse_number_properties(perfect)
            perfect_data.append({
                'Perfect Number': perfect,
                'Exponent p': p,
                'Divisor Count': props['divisor_count'],
                'Sigma': props['sigma']
            })
        
        df = pd.DataFrame(perfect_data)
        
        # Create a table-like visualisation
        axes[1, 1].axis('tight')
        axes[1, 1].axis('off')
        table = axes[1, 1].table(cellText=df.values, colLabels=df.columns,
                                cellLoc='center', loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1.2, 1.5)
        axes[1, 1].set_title('Perfect Number Properties')
    
    # 6. Theoretical Growth Comparison
    # Compare actual perfect numbers with theoretical growth 2^(2p-1)
    theoretical = [2**(2*p-1) for p in exponents]
    
    axes[1, 2].plot(exponents, values, 'bo-', label='Actual Perfect Numbers', linewidth=2)
    axes[1, 2].plot(exponents, theoretical, 'r--', label='Theoretical 2^(2p-1)', linewidth=2)
    axes[1, 2].set_yscale('log')
    axes[1, 2].set_xlabel('Exponent p')
    axes[1, 2].set_ylabel('Value (log scale)')
    axes[1, 2].set_title('Actual vs Theoretical Growth')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle(f'Perfect Number Analysis (p ≤ {max_p})', fontsize=16, y=1.02)
    plt.show()
    
    # Print summary statistics
    print(f"\n📊 Perfect Number Statistics (p ≤ {max_p}):")
    print("=" * 50)
    print(f"Total perfect numbers found: {len(perfects)}")
    print(f"Largest perfect number: {values[-1]:.6e}")
    print(f"Number of digits in largest: {len(str(values[-1]))}")
    if len(values) > 1:
        avg_growth = np.mean([values[i] / values[i-1] for i in range(1, len(values))])
        print(f"Average growth ratio: {avg_growth:.2f}")
    print(f"Corresponding Mersenne exponents: {list(exponents)}")

# Create visualisations
create_perfect_number_visualisations(20)

## Number Classification Explorer

Explore perfect, abundant, and deficient numbers:

In [ ]:
# Interactive number classification explorer
number_input = widgets.IntText(
    value=28,
    description='Number:',
    tooltip='Enter a number to analyse'
)

analyse_button = widgets.Button(
    description='Analyse Number',
    button_style='info',
    tooltip='Analyse the mathematical properties of the number'
)

range_start = widgets.IntText(
    value=1,
    description='Range Start:',
    tooltip='Start of range for batch analysis'
)

range_end = widgets.IntText(
    value=100,
    description='Range End:',
    tooltip='End of range for batch analysis'
)

analyse_range_button = widgets.Button(
    description='Analyse Range',
    button_style='success',
    tooltip='Analyse all numbers in the specified range'
)

analysis_output = widgets.Output()

def analyse_single_number(b):
    with analysis_output:
        clear_output()
        n = number_input.value
        
        if n <= 0:
            print("❌ Please enter a positive integer.")
            return
        
        props = analyse_number_properties(n)
        
        print(f"🔍 Analysis of {n}:")
        print("=" * 40)
        
        # Basic classification
        if props['is_perfect']:
            classification = "✨ PERFECT"
            emoji = "🎯"
        elif props['is_abundant']:
            classification = "📈 ABUNDANT"
            emoji = "⬆️"
        else:
            classification = "📉 DEFICIENT"
            emoji = "⬇️"
        
        print(f"{emoji} Classification: {classification}")
        print(f"📊 Abundance: {props['abundance']} (σ'(n) - n)")
        print(f"🔢 Divisor count: {props['divisor_count']}")
        print(f"➕ Sum of proper divisors: {props['sigma_proper']}")
        print(f"🌟 Sum of all divisors: {props['sigma']}")
        
        # Prime factorisation
        print(f"\n🧮 Prime Factorisation:")
        for prime, power in props['prime_factors'].items():
            if power == 1:
                print(f"   {prime}")
            else:
                print(f"   {prime}^{power}")
        
        # Divisors
        print(f"\n📋 All Divisors: {props['divisors']}")
        print(f"📋 Proper Divisors: {props['proper_divisors']}")
        
        # Special properties
        print(f"\n🔍 Special Properties:")
        if sympy.isprime(n):
            print(f"   • Prime number")
        if n > 1 and all(sympy.isprime(p) for p in props['prime_factors'].keys()):
            if len(props['prime_factors']) == 1:
                print(f"   • Prime power")
            else:
                print(f"   • Semiprime" if len(props['prime_factors']) == 2 else "   • Composite with distinct prime factors")
        
        # Check if it's a power of 2
        if n > 0 and (n & (n - 1)) == 0:
            print(f"   • Power of 2: 2^{int(log2(n))}")

def analyse_number_range(b):
    with analysis_output:
        clear_output()
        start = range_start.value
        end = range_end.value
        
        if start <= 0 or end <= start or end - start > 1000:
            print("❌ Please enter a valid range (positive integers, start < end, range ≤ 1000).")
            return
        
        print(f"🔍 Analysing numbers from {start} to {end}...\n")
        
        perfect_nums = []
        abundant_nums = []
        deficient_nums = []
        
        for n in range(start, end + 1):
            props = analyse_number_properties(n)
            if props['is_perfect']:
                perfect_nums.append(n)
            elif props['is_abundant']:
                abundant_nums.append(n)
            else:
                deficient_nums.append(n)
        
        total = end - start + 1
        
        print(f"📊 Classification Summary:")
        print("=" * 30)
        print(f"✨ Perfect numbers: {len(perfect_nums)} ({len(perfect_nums)/total*100:.1f}%)")
        print(f"📈 Abundant numbers: {len(abundant_nums)} ({len(abundant_nums)/total*100:.1f}%)")
        print(f"📉 Deficient numbers: {len(deficient_nums)} ({len(deficient_nums)/total*100:.1f}%)")
        
        if perfect_nums:
            print(f"\n🎯 Perfect numbers found: {perfect_nums}")
        
        if len(abundant_nums) <= 20:
            print(f"\n📈 Abundant numbers: {abundant_nums}")
        else:
            print(f"\n📈 First 20 abundant numbers: {abundant_nums[:20]}...")
        
        # Create a simple visualisation
        if total <= 100:  # Only for smaller ranges
            categories = ['Perfect', 'Abundant', 'Deficient']
            counts = [len(perfect_nums), len(abundant_nums), len(deficient_nums)]
            colors = ['gold', 'lightcoral', 'lightblue']
            
            plt.figure(figsize=(10, 6))
            
            plt.subplot(1, 2, 1)
            plt.bar(categories, counts, color=colors, alpha=0.7, edgecolor='black')
            plt.title(f'Number Classification ({start}-{end})')
            plt.ylabel('Count')
            
            # Add count labels
            for i, count in enumerate(counts):
                plt.text(i, count + 0.1, str(count), ha='center', va='bottom')
            
            plt.subplot(1, 2, 2)
            plt.pie(counts, labels=categories, colors=colors, autopct='%1.1f%%', startangle=90)
            plt.title('Distribution of Number Types')
            
            plt.tight_layout()
            plt.show()

# Import log2 for power of 2 check
from math import log2

analyse_button.on_click(analyse_single_number)
analyse_range_button.on_click(analyse_number_range)

# Display the classification explorer
classification_widget = widgets.VBox([
    widgets.HTML("<h2>🔍 Number Classification Explorer</h2>"),
    widgets.HTML("<h3>Single Number Analysis</h3>"),
    number_input,
    analyse_button,
    widgets.HTML("<h3>Range Analysis</h3>"),
    widgets.HBox([range_start, range_end]),
    analyse_range_button,
    analysis_output
])

display(classification_widget)

# Trigger initial analysis
analyse_single_number(None)

## Odd Perfect Number Investigation

Explore the mystery of odd perfect numbers and their theoretical constraints:

In [ ]:
def investigate_odd_perfect_constraints():
    """Investigate known constraints on odd perfect numbers."""
    print("🔍 Known Constraints on Odd Perfect Numbers:")
    print("=" * 55)
    
    constraints = [
        "1. Must be greater than 10^300 (Nielsen, 2003)",
        "2. Must have at least 75 prime factors (Nielsen, 2003)",
        "3. Must have at least 9 distinct prime factors (Grün, 1952)",
        "4. Largest prime factor must exceed 10^8 (Goto & Ohno, 2008)",
        "5. Second largest prime factor must exceed 10^4 (Hagis, 1983)",
        "6. Third largest prime factor must exceed 100 (Hagis, 1983)",
        "7. Must be congruent to 1 (mod 12) or one of several other residues",
        "8. Cannot be divisible by 105 = 3 × 5 × 7",
        "9. Must be a perfect square multiplied by a prime or twice a prime"
    ]
    
    for constraint in constraints:
        print(f"   {constraint}")
    
    print(f"\n💡 Despite extensive research, no odd perfect number has ever been found!")
    print(f"   This remains one of the oldest unsolved problems in mathematics.")

def test_odd_perfect_conditions(n):
    """
    Test if a number satisfies some necessary conditions for odd perfect numbers.
    Note: These are necessary but not sufficient conditions.
    """
    if n % 2 == 0:
        return False, "Number is even"
    
    if not basic_conditions_odd(n):
        return False, "Fails basic modular conditions"
    
    if n % 105 == 0:
        return False, "Divisible by 105 = 3×5×7"
    
    factors = sympy.factorint(n)
    primes = sorted(factors.keys(), reverse=True)
    
    if len(primes) < 9:
        return False, f"Has only {len(primes)} distinct prime factors (need ≥9)"
    
    if primes[0] <= 10**8:
        return False, f"Largest prime factor {primes[0]} ≤ 10^8"
    
    if len(primes) > 1 and primes[1] <= 10**4:
        return False, f"Second largest prime factor {primes[1]} ≤ 10^4"
    
    if len(primes) > 2 and primes[2] <= 100:
        return False, f"Third largest prime factor {primes[2]} ≤ 100"
    
    return True, "Passes basic necessary conditions"

def simulate_odd_perfect_search():
    """Simulate search for odd perfect numbers in a small range (educational)."""
    print("\n🔍 Simulating Odd Perfect Number Search (Educational):")
    print("=" * 60)
    print("Note: Real searches require massive computational resources!\n")
    
    # Test some odd numbers and show why they fail
    test_numbers = [15, 21, 33, 45, 63, 75, 105, 135, 165, 195]
    
    print("Testing small odd numbers (for demonstration):")
    print("-" * 50)
    
    for n in test_numbers:
        props = analyse_number_properties(n)
        passes, reason = test_odd_perfect_conditions(n)
        
        status = "✅" if props['is_perfect'] else "❌"
        print(f"{status} {n:3d}: σ'({n}) = {props['sigma_proper']:3d}, "
              f"{'PERFECT' if props['is_perfect'] else 'not perfect'} - {reason}")
    
    print(f"\n📊 Search Results:")
    print(f"   Perfect numbers found: 0")
    print(f"   Numbers tested: {len(test_numbers)}")
    print(f"   Computational complexity: O(√n) per number")
    
    # Show the theoretical scale of the problem
    print(f"\n🚀 Scale of the Real Problem:")
    print(f"   • Must search numbers > 10^300")
    print(f"   • That's a 1 followed by 300 zeros!")
    print(f"   • Current estimates suggest any odd perfect number")
    print(f"     would have more than 500 digits")
    print(f"   • Computational time: centuries even with supercomputers")

# Run the odd perfect number investigation
investigate_odd_perfect_constraints()
simulate_odd_perfect_search()

# Create a visualisation of the constraint hierarchy
def visualise_odd_perfect_constraints():
    """Visualise the constraint hierarchy for odd perfect numbers."""
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    
    # Create a hierarchical constraint visualisation
    constraints = [
        ("All Odd Numbers", 10**10, 'lightblue'),
        ("Pass Modular Tests", 10**9, 'lightgreen'),
        ("≥9 Prime Factors", 10**8, 'yellow'),
        ("Largest Prime > 10^8", 10**7, 'orange'),
        ("2nd Largest > 10^4", 10**6, 'red'),
        ("Must be > 10^300", 1, 'darkred')
    ]
    
    y_positions = list(range(len(constraints)))
    widths = [count for _, count, _ in constraints]
    colors = [color for _, _, color in constraints]
    labels = [label for label, _, _ in constraints]
    
    bars = ax.barh(y_positions, widths, color=colors, alpha=0.7, edgecolor='black')
    
    ax.set_yticks(y_positions)
    ax.set_yticklabels(labels)
    ax.set_xlabel('Approximate Count (Log Scale)')
    ax.set_xscale('log')
    ax.set_title('Odd Perfect Number Constraint Hierarchy\n(How constraints eliminate candidates)')
    ax.grid(True, alpha=0.3)
    
    # Add text annotations
    for i, (bar, width) in enumerate(zip(bars, widths)):
        ax.text(width * 1.1, bar.get_y() + bar.get_height()/2, 
                f'{width:.0e}', va='center', ha='left')
    
    plt.tight_layout()
    plt.show()

visualise_odd_perfect_constraints()

## Mathematical Exploration Exercises

Interactive exercises to deepen understanding:

In [ ]:
def mathematical_exercises():
    """Present mathematical exercises related to perfect numbers."""
    print("📚 Mathematical Exercises: Perfect Numbers")
    print("=" * 50)
    
    exercises = [
        {
            'question': "Verify that 496 is a perfect number by finding all its divisors.",
            'answer': lambda: analyse_number_properties(496),
            'explanation': "496 = 2^4 × 31, and 31 is prime, so 496 = 2^(5-1) × (2^5 - 1) is perfect."
        },
        {
            'question': "Find the next perfect number after 28.",
            'answer': lambda: generate_perfect(10)[2][1] if len(generate_perfect(10)) > 2 else None,
            'explanation': "The next perfect number is 496, corresponding to p=5 in the Euclid-Euler formula."
        },
        {
            'question': "What is the relationship between perfect numbers and triangular numbers?",
            'answer': None,
            'explanation': "Every even perfect number is a triangular number! Perfect number 2^(p-1)(2^p-1) is the ((2^p-1))th triangular number."
        }
    ]
    
    for i, exercise in enumerate(exercises, 1):
        print(f"\n🧮 Exercise {i}:")
        print(f"   {exercise['question']}")
        
        if exercise['answer']:
            if callable(exercise['answer']):
                result = exercise['answer']()
                if isinstance(result, dict):
                    print(f"\n   💡 Solution:")
                    print(f"      Number: {result['number']}")
                    print(f"      Proper divisors: {result['proper_divisors']}")
                    print(f"      Sum: {result['sigma_proper']}")
                    print(f"      Is perfect: {result['is_perfect']}")
                else:
                    print(f"\n   💡 Answer: {result}")
        
        print(f"\n   📖 Explanation: {exercise['explanation']}")
    
    # Additional exploration
    print(f"\n🔍 Further Exploration:")
    print(f"1. Investigate the binary representation of perfect numbers")
    print(f"2. Explore the relationship between perfect numbers and Mersenne primes")
    print(f"3. Study the gaps between consecutive perfect numbers")
    print(f"4. Research the history of perfect number discoveries")
    
def demonstrate_perfect_number_patterns():
    """Demonstrate interesting patterns in perfect numbers."""
    perfects = generate_perfect(15)
    
    print(f"\n🎨 Patterns in Perfect Numbers:")
    print("=" * 40)
    
    # Pattern 1: Binary representation
    print(f"\n1️⃣ Binary Representations:")
    for p, perfect in perfects[:4]:
        binary = bin(perfect)[2:]  # Remove '0b' prefix
        print(f"   {perfect:>6} = {binary}")
    
    # Pattern 2: Digit sums
    print(f"\n2️⃣ Digital Roots:")
    for p, perfect in perfects[:6]:
        digit_sum = sum(int(d) for d in str(perfect))
        while digit_sum >= 10:
            digit_sum = sum(int(d) for d in str(digit_sum))
        print(f"   {perfect}: digital root = {digit_sum}")
    
    # Pattern 3: Last digits
    print(f"\n3️⃣ Last Digit Pattern:")
    last_digits = [perfect % 10 for _, perfect in perfects[:8]]
    print(f"   Last digits: {last_digits}")
    print(f"   Pattern: Perfect numbers > 6 end in 6 or 28!")
    
    # Pattern 4: Triangular number connection
    print(f"\n4️⃣ Triangular Number Connection:")
    for p, perfect in perfects[:4]:
        # Find which triangular number this perfect number is
        # T_n = n(n+1)/2, so n = (-1 + sqrt(1 + 8*T))/2
        n = int((-1 + (1 + 8*perfect)**0.5) / 2)
        triangular = n * (n + 1) // 2
        if triangular == perfect:
            print(f"   {perfect} = T_{n} (the {n}th triangular number)")
    
    print(f"\n✨ Every even perfect number is also a triangular number!")

# Run the exercises and demonstrations
mathematical_exercises()
demonstrate_perfect_number_patterns()

## Summary

This notebook has provided a comprehensive exploration of perfect numbers:

### 🔢 Key Concepts Covered:
- **Even Perfect Numbers**: Generation using the Euclid-Euler theorem
- **Number Classification**: Perfect, abundant, and deficient numbers
- **Mersenne Primes**: Their crucial role in perfect number generation
- **Odd Perfect Numbers**: The ongoing mathematical mystery
- **Mathematical Properties**: Patterns, growth rates, and special characteristics

### 📊 Educational Value:
- ✅ **Interactive Exploration**: Widgets for hands-on number analysis
- ✅ **Visual Learning**: Comprehensive charts and visualisations
- ✅ **Mathematical Depth**: Theoretical foundations and proofs
- ✅ **Historical Context**: Ancient problems and modern research
- ✅ **Computational Awareness**: Understanding algorithmic complexity

### 🧮 Key Mathematical Insights:
1. **Euclid-Euler Theorem**: Foundation for all known perfect numbers
2. **Mersenne Primes**: Critical for perfect number generation
3. **Growth Patterns**: Exponential growth with fascinating regularities
4. **Unsolved Mystery**: Odd perfect numbers remain elusive
5. **Computational Limits**: Modern constraints on mathematical exploration

### 🚀 Advanced Applications:
- **Number Theory**: Deep connections to prime number theory
- **Computational Mathematics**: Algorithm design and optimisation
- **Cryptography**: Applications in security and encryption
- **Mathematical Research**: Ongoing investigations and discoveries
- **Educational Tools**: Interactive learning and exploration

### 🔬 Research Frontiers:
- **Odd Perfect Numbers**: The ultimate unsolved problem
- **Mersenne Prime Search**: Distributed computing projects
- **Generalised Perfect Numbers**: Extensions and variations
- **Computational Limits**: Pushing the boundaries of calculation

Perfect numbers represent a beautiful intersection of pure mathematics and computational exploration, demonstrating how ancient mathematical curiosities continue to challenge and inspire modern research. The Euclid-Euler theorem provides a complete characterisation of even perfect numbers, whilst the mystery of odd perfect numbers remains one of mathematics' most enduring puzzles.

The study of perfect numbers showcases the power of mathematical proof, the importance of computational verification, and the ongoing nature of mathematical discovery. Whether odd perfect numbers exist or not, their investigation has led to profound insights into number theory, prime numbers, and the nature of mathematical truth itself.